# Creating a modified version of the AWRA-L model

In this example, we will create a local copy of AWRAL, and modify this copy to replace the existing (modelled) albedo with input data (eg satellite derived)

#### The copy of awral is renamed to 'albmod' since we are modifying for albedo...

In [ ]:
# get directory location of awral module
# copy it to current working directory
from awrams.models import awral 
import os
cwd=os.getcwd()
!mkdir testmodules
path =  os.path.dirname(os.path.dirname(awral.__file__))  
# change working directory (%cd) to  awra module path  ($path)
# $path converts path string to the literal name without the '' around '...packages/awrams/models'
outfoldername = cwd+'/testmodules/albmod/'
%cd $path
!cp -r awral  $outfoldername
%cd $cwd
# should now have albmod folder in current working directory
!ls

In [ ]:
# list the files in folder testmodules - should now see albmod copy of awral
!ls testmodules

### Note: PYTHONPATH and creating your own modules

- It is Good Python practice to have a specified directory that you store your own modules (and not have this within other packages).
- You then point Python environment at these modules by specifying them within PYTHONPATH: an environment variable.
- In this case we have created a module within the folder testmodules/albmod 
- You append the folder to PYTHONPATH environment variable from the command line before starting your environment
- First 'export' the testmodules path into PYTHONPATH by typing

           export PYTHONPATH=$PYTHONPATH:/path/to/extra/modules

- where /path/to/extra/modules is the path to testmodules
- by doing this you will be able to use these modules with the rest of AWRA CMS (e.g. in calibration, simulation)



In [ ]:
# PYTHONPATH exists specifically for user added folders
# view current listing of PYTHONPATH by using !echo $PYTHONPATH - will return nothing
# it'll be blank by default (ie conda already 'knows' where the Python related modules are); 
# it will return something if you have added to PYTHONPAT previously
!echo $PYTHONPATH

## Required modifications

There are 3 modifications required in this instance.<br>

1. ***Add albedo input in model_inputs.json*** - add "alb": "Surface albedo" variable to model_inputs.json
2. ***Modify core code (awral_t.c)*** - alter awral_t.c to remove existing albedo calculation and use input value
3. ***Populate the input mapping*** - 


### 1. Add albedo input in [model_inputs.json]
[model_inputs.json]: ../../../edit/Training/Advanced/testmodules/albmod/data/model_inputs.json
The AWRA-L model uses a JSON file to populate it's input keys (ie the values returned by get_input_parameters())<br>
This is stored in data/model_inputs.json<br>
New model inputs need to be added to this file first

In [ ]:
# list files available within testmodules/albmod/data folder to check model_inputs.json file is there
# should see DefaultParameters.json	model_inputs.json  spatial_parameters.h5
!ls testmodules/albmod/data

### Modify albmod/data/[model_inputs.json] INPUTS_CELL to include an albedo variable ***alb***


There are various types of inputs specified in this file:

  - ***STATES_CELL***  - model states that have a single value in a grid/catchment cell (do not differ by HRU)
  - ***STATES_HRU***   - model states that have a different values on a HRU basis value in a grid/catchment cell 
  - ***INPUTS_CELL***  - model inputs that have a single value in a grid/catchment cell (do not differ by HRU)
  - ***INPUTS_HRU***   - model inputs that have a different values on a HRU basis value in a grid/catchment cell
  - ***INPUTS_HYPSO*** - inputs related to the Hypsometric curves
  
The type of data (ie scalar, spatial, or forcing (timeseries)) is computed from the input map, so does not need to be specified here

Add the following line (at line 13) to the INPUTS_CELL section of model_inputs.json

    "alb": "Surface albedo",


#### Existing [model_inputs.json]

    1:   {
    2:    "STATES_CELL": {
    3:        "sg": "Groundwater storage (mm)",
    4:        "sr": "Surface storage (mm)"
    5:    },
    6:    "STATES_HRU": {
    7:        "mleaf": "Vegetation index",
    8:        "s0": "Top soil moisture (mm)",
    9:        "sd": "Deep soil moisture (mm)",
    10:        "ss": "Shallow soil moisture (mm)"
    11:    },
    12:    "INPUTS_CELL": {
    13:        "avpt": "Vapour pressure",
    14:        "k0sat": "Hydraulic saturation (top)",
    15:        "k_gw": "Groundwater drainage coefficient",
    ...
    35:    },
    36:    "INPUTS_HRU": {
    37:        "alb_dry": "Dry Soil Albedo",
    38:        "alb_wet": "Wet Soil Albedo",
    39:        "cgsmax": "Conversion Coefficient From Vegetation Photosynthetic Capacity Index to Maximum Stomatal Conductance",
    ...    
    58:    }, 
    59:    "INPUTS_HYPSO": {
    60:        "height": "height",
    61:        "hypsperc": "hypsperc",
    62:        "ne": "ne"
    62:    }



#### [model_inputs.json] With new "alb": "Surface albedo" input added   

    12:    "INPUTS_CELL": {
    13:        "alb": "Surface albedo",
    14:        "avpt": "Vapour pressure",
    15:        "k0sat": "Hydraulic saturation (top)",
    16:        "k_gw": "Groundwater drainage coefficient",
    ...

[model_inputs.json]: ../../../edit/Training/Advanced/testmodules/albmod/data/model_inputs.json

In [ ]:
# %load albmod/data/model_inputs.json

# 2. Modify core code ([awral_t.c])

### Remove existing code block

Comment out the section of the main model code ([awral_t.c]) that contains the current albedo calculation<br>
The dynamic compiler will automatically generate the code for receiving the new input data; this is all you need to do!


/albmod/core/[awral_t.c]

    223: double alb_veg = 0.452 * vc;
    224: double alb_soil = alb_wet + (alb_dry - alb_wet) * exp(-w0 / w0ref_alb);
    225: double alb = fveg * alb_veg + fsoil * alb_soil;
    226: double rsn = (1.0 - alb) * rgeff;

Comment out old albedo calculation code by using // ...

    223: //double alb_veg = 0.452 * vc;
    224: //double alb_soil = alb_wet + (alb_dry - alb_wet) * exp(-w0 / w0ref_alb);
    225: //double alb = fveg * alb_veg + fsoil * alb_soil;
    226: double rsn = (1.0 - alb) * rgeff;
    
[awral_t.c]: ../../../edit/Training/Advanced/testmodules/albmod/core/awral_t.c

In [ ]:
# %load albmod/core/awral_t.c

## 3. Populate input mapping

The model is ready to receive a new input, 'alb'; we now need to specify where this value comes from<br>

For testing purposes, let's just start off supplying a constant value.

#### Modify get_default_mapping() function in albmod/[model.py]

Add the following line to the get_default_mapping() function within model.py (starting line 58)

    mapping['alb'] = nodes.const(0.3)

similar to         
        
    128:  mapping['pair'] = nodes.const(97500.)

You can change this value or perform other data transforms at runtime; the main reason for editing get_default_mapping is so that it has a valid default value.

[model.py]: ../../../edit/Training/Advanced/testmodules/albmod/model.py

In [ ]:
# %load albmod/model.py

### Run the modified model

In [ ]:
from awrams.utils.mapping_types import gen_coordset
from awrams.utils import datetools as dt
from awrams.utils import extents
from awrams.simulation.ondemand import OnDemandSimulator
from awrams.utils.nodegraph import nodes,graph

In [ ]:
# Now we can import from this awral model copy testmodules.albmod
from testmodules.albmod import model
m = model.AWRALModel()

In [ ]:
# Check the input keys; 'alb' is now here
# Note that we still have some redundant inputs (alb_dry, alb_wet)
# These can be removed now
sorted(m.get_input_keys())

In [ ]:
# Examine the input mapping to see if 'alb' variable is there
imap = m.get_default_mapping()
imap['alb']

In [ ]:
# Create a function change_path_to_forcing() to change from the default paths to  
def change_path_to_forcing(imap):
    from awrams.utils.nodegraph import nodes
    from os.path import join
    from os import getcwd
    from awrams.models.settings import TRAINING_DATA_PATH

    # location of registered user data in the Training folder
    data_path = TRAINING_DATA_PATH + '/climate/BOM_climate/'

    FORCING = {
        'tmin': ('temp_min*.nc','temp_min_day',data_path + 'tmin/'),
        'tmax': ('temp_max*.nc','temp_max_day',data_path + 'tmax/'),
        'precip': ('rain_day*.nc','rain_day',data_path + 'rr/'),
        'solar': ('solar*.nc','solar_exposure_day',data_path + 'rad/') #,
        #'vprp':('vapour_pressure*.nc', 'vapour_pressure', vprp_data_path + 'vapour_pressure/'), #h09
        #'wind':('wind*.nc', 'wind', wind_data_path)
    }
    for k,v in FORCING.items():
        imap[k+'_f'] = nodes.forcing_from_ncfiles(v[2],v[0],v[1])

change_path_to_forcing(imap)

In [ ]:
# We can easily verify that the core model changes are working by adding 'alb' as an _output_ from the model
# Note the list below does not contain 'alb
m.OUTPUTS

In [ ]:
m.OUTPUTS['OUTPUTS_AVG'].append('alb')
m.OUTPUTS

In [ ]:
!pwd

In [ ]:
# Set up a simulation

# This will trigger a model rebuild (when the OnDemandSimulator calls get_runner())

sim = OnDemandSimulator(m,imap)

In [ ]:
e = extents.get_default_extent().ioffset[200,200]
p = dt.dates('jan 2011')

In [ ]:
results = sim.run(p,e)

In [ ]:
import pandas as pd

In [ ]:
# Examine the data; alb should have a constant value of 0.30
%matplotlib inline
pd.Series(results['alb'][:,0],p).plot()

### Further excercises

Change the input mapping for 'alb' to point to use real data instead of a const